In [1]:
import pandas
import numpy

## Les données du problème

In [2]:
projets = [
    [-14, 90, 0, 40, 100],
    [129, 100, 0, 0, 0],
    [-10, 50, 0, 10, 100],
    [44, 90, 0, 5, 20],
    [-14, 100, 0, 20, 40]
]
p = [50, 24, 1, 24, 20]
q = [25, 16, 0, 12, 10]
importance = [1, 1, 1, 1, 1]
veto = [100, 60, 2, 48, 90]

In [3]:
projects_names = ['P' + str(i + 1) for i in range(len(projets))]
criterias_names = ['C' + str(i + 1) for i in range(len(projets[0]))]

df = pandas.DataFrame(data=projets + [p, q, importance, veto],
                      columns=criterias_names,
                      index=projects_names + ['Préférence', 'Indifférence', 'Importance', 'Veto'])

df

,C1,C2,C3,C4,C5
P1,-14,90,0,40,100
P2,129,100,0,0,0
P3,-10,50,0,10,100
P4,44,90,0,5,20
P5,-14,100,0,20,40
Préférence,50,24,1,24,20
Indifférence,25,16,0,12,10
Importance,1,1,1,1,1
Veto,100,60,2,48,90


## Matrice de concordance

In [4]:
k = df.loc["Importance"].sum()

concordance = pandas.DataFrame(index=projects_names,
                               columns=projects_names)

for a in projects_names:
    for b in projects_names:
        s = 0
        for j in criterias_names:
            if df.loc[a][j] >= df.loc[b][j] - df.loc["Indifférence"][j]:
                indice_concordance = 1
            elif df.loc[a][j] <= df.loc[b][j] - df.loc["Préférence"][j]:
                indice_concordance = 0
            else:
                indice_concordance = (df.loc["Préférence"][j] + df.loc[a][j] - df.loc[b][j]) / (df.loc["Préférence"][j] - df.loc["Indifférence"][j])
            s += df.loc["Importance"][j] * indice_concordance
        concordance.loc[a][b] = 1/k * s

concordance

,P1,P2,P3,P4,P5
P1,1,0.8,1,0.8,1
P2,0.6,1,0.8,0.8,0.666667
P3,0.6,0.6,1,0.6,0.8
P4,0.6,0.8,0.8,1,0.75
P5,0.666667,0.8,0.8,0.8,1


## Matrice de crédibilité

In [5]:
credibilite = pandas.DataFrame(index=projects_names,
                               columns=projects_names)

for a in projects_names:
    for b in projects_names:
        discordances = pandas.Series(index = criterias_names, dtype=int)
        for j in criterias_names:
            if df.loc[a][j] >= df.loc[b][j] - df.loc["Préférence"][j]:
                discordances[j] = 0
            elif df.loc[a][j] <= df.loc[b][j] - df.loc["Veto"][j]:
                discordances[j] = 1
            else:
                discordances[j] = (df.loc[b][j] - df.loc[a][j] - df.loc["Préférence"][j]) / (df.loc["Veto"][j] - df.loc["Préférence"][j])
                
        J = discordances[discordances > concordance.loc[a][b]].index

        if len(J) == 0:
            credibilite.loc[a][b] = concordance.loc[a][b]
        else:
            credibilite.loc[a][b] = concordance.loc[a][b] * ((1 - discordances[J]) / (1 - concordance.loc[a][b])).product()

credibilite

,P1,P2,P3,P4,P5
P1,1,0,1,0.8,1
P2,0,1,0,0.8,0.666667
P3,0.6,0,1,0.6,0.8
P4,0.6,0.8,0.8,1,0.75
P5,0.666667,0,0.8,0.8,1


## Distillation ascendante

In [6]:
alpha = 0.4 # normalement mettre 0.3
beta = -0.15

order = []

credibilite2 = credibilite.copy()

for i in range(0,len(credibilite)):
    lamba = numpy.amax(credibilite2.to_numpy())
    s_lamba = alpha + beta * lamba
    qualification = pandas.Series(index = credibilite2.columns, dtype=int)
    for a in credibilite2.columns:
        for b in credibilite2.index.values:
            if a != b and credibilite2.loc[a][b] >= lamba - s_lamba \
            and abs(credibilite2.loc[a][b] - credibilite2.loc[b][a]) <= s_lamba:
                #print("{} -> {}".format(a, b))
                qualification[a]+=1
                qualification[b]-=1

    max_qual = qualification.idxmax()
    # On arrête si on a pas de meilleur TODO voir comment faire
    if (qualification == 0).all():
        break
    credibilite2 = credibilite2.drop(max_qual, 0)
    credibilite2 = credibilite2.drop(max_qual, 1)
    order.append(max_qual)

order

['P1', 'P4']